In [13]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from ganblr.utils import get_demo_data
import warnings
warnings.filterwarnings('ignore')

df = get_demo_data('adult') #Optional: adult or car
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,6,7,0,9,2,4,1,1,4,1,13,0,2,39,0
1,7,6,0,9,2,2,4,0,4,1,0,0,0,39,0
2,6,4,0,11,5,0,6,1,4,1,0,0,2,39,0
3,7,4,0,1,0,2,6,0,2,1,0,0,2,39,0
4,4,4,0,9,2,2,10,5,2,0,0,0,2,5,0


In [14]:
data = OrdinalEncoder(dtype=int).fit_transform(df)
x, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

In [15]:
from ganblr import GANBLR
model = GANBLR()
model.fit(X_train, y_train, k=0, batch_size=32, epochs = 10)

int32 int32
764/764 [==============================] - 1s 662us/step - loss: 3.2187 - accuracy: 0.8720


In [16]:
acc_score_lr = model.evaluate(X_test, y_test, model='lr')
acc_score_mlp = model.evaluate(X_test, y_test, model='mlp')
acc_score_rf = model.evaluate(X_test, y_test, model='rf')

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
X_train_ohe = ohe.fit_transform(X_train)
X_test_ohe = ohe.transform(X_test)
trtr_score_lr  = LogisticRegression().fit(X_train_ohe, y_train).score(X_test_ohe, y_test)
trtr_score_rf  = RandomForestClassifier().fit(X_train, y_train).score(X_test, y_test)
trtr_score_mlp = MLPClassifier().fit(X_train_ohe, y_train).score(X_test_ohe, y_test)

In [20]:
import pandas as pd
df_evaluate = pd.DataFrame([
    ['TSTR', acc_score_lr, acc_score_rf, acc_score_mlp],
    ['TRTR', trtr_score_lr,trtr_score_rf,trtr_score_mlp]
], columns=['Evaluate Item', 'LR', 'RF', 'MLP'])
df_evaluate
#df_evaluate.set_index('Evaluate Item')

,Evaluate Item,LR,RF,MLP
0,TSTR,0.870276,0.834691,0.854756
1,TRTR,0.873633,0.851767,0.859342


In [3]:
from ganblr import GANBLRPP

from pandas import DataFrame, read_csv
df = read_csv('../uci-datasets/raw_csv/adult.csv', index_col=0)

In [4]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39.0,5,77516.0,0,13.0,2,8,3,0,1,2174.0,0.0,40.0,0,1
1,50.0,1,83311.0,0,13.0,0,4,2,0,1,0.0,0.0,13.0,0,1
2,38.0,0,215646.0,3,9.0,1,6,3,0,1,0.0,0.0,40.0,0,1
3,53.0,0,234721.0,2,7.0,0,6,2,4,1,0.0,0.0,40.0,0,1
4,28.0,0,338409.0,0,13.0,0,5,0,4,0,0.0,0.0,40.0,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39.0,0,215419.0,0,13.0,1,5,3,0,0,0.0,0.0,36.0,0,1
48838,64.0,8,321403.0,3,9.0,4,14,4,4,1,0.0,0.0,40.0,0,1
48839,38.0,0,374983.0,0,13.0,0,5,2,0,1,0.0,0.0,50.0,0,1
48840,44.0,0,83891.0,0,13.0,1,8,1,1,1,5455.0,0.0,40.0,0,1


In [5]:
import numpy as np
numerical_columns = np.argwhere(df.dtypes.values.ravel() == float).ravel()

In [6]:
numerical_columns

array([ 0,  2,  4, 10, 11, 12], dtype=int64)

In [7]:
ganblrpp = GANBLRPP(numerical_columns)

In [8]:
from sklearn.model_selection import train_test_split
x, y = df.values[:,:-1], df.values[:,-1]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

In [9]:
ganblrpp.fit(X_train, y_train, epochs=50)

c:\Users\mikaya\anaconda3\lib\site-packages\sklearn\mixture\_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(
c:\Users\mikaya\anaconda3\lib\site-packages\sklearn\mixture\_base.py:277: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn(


int32 int32
764/764 [==============================] - 0s 437us/step


c:\Repos\ganblr-opensource\ganblr\models\ganblr.py:62: RuntimeWarning: divide by zero encountered in log
  ls = np.mean(-np.log(np.subtract(1, prob_fake)))


764/764 [==============================] - 2s 655us/step - loss: 4.5006 - accuracy: 0.8650


In [11]:
acc_score_lr  = ganblrpp.evaluate(X_test, y_test, model='lr')
acc_score_mlp = ganblrpp.evaluate(X_test, y_test, model='mlp')
acc_score_rf  = ganblrpp.evaluate(X_test, y_test, model='rf')

  0%|          | 0/15 [00:00<?, ?it/s]

ValueError: Found unknown categories [8.0, 24.0, 32.0] in column 0 during transform

In [12]:
syn_data = ganblrpp.sample()

Step 1 of 2: Sampling discrete data from GANBLR.


  0%|          | 0/15 [00:00<?, ?it/s]

step 2 of 2: Sampling numerical data.


TypeError: 'list' object cannot be interpreted as an integer